# Лабораторная работа №3 
## Пятнашки 


In [1]:
from copy import deepcopy

class PuzzleGame:
    def __init__(self, initial_state):
        self.initial_state = initial_state
        self.matrix = []
#ключи
        self.MOVE = {
            'UP'    :   (-1, 0),
            'DOWN'  :   (1, 0), 
            'LEFT'  :   (0, -1),
            'RIGHT' :   (0, 1)
            }

    def get_matrix(self):
        matrix = ''
        for row in self.matrix:
            matrix += str(row) + '\n'

        return matrix

    def __str__(self):
        return self.get_matrix()
#перемещения возможные
    def check_moves(self, prev_move):
        if prev_move == None:
            return tuple(self.MOVE.keys())

        if prev_move == 'UP':
            return ('UP', 'LEFT', 'RIGHT')

        if prev_move == 'DOWN':
            return ('DOWN', 'LEFT', 'RIGHT')

        if prev_move == 'LEFT':
            return ('UP', 'DOWN', 'LEFT')

        if prev_move == 'RIGHT':
            return ('UP', 'DOWN', 'RIGHT')
#можно ли переместить
#НЕ выход за матрицу 
    def move_allowed(self, place, move):
        row, col = self.new_place(place, move)
        if row > 0 and row < 5 and col > 0 and col < 5:
            return True

        else:
            return False
#новая координата нуля 
    def new_place(self, place, move):
        return (place[0]+move[0], place[1]+move[1])
    
#копируем матрицу 
    def apply_move(self, place, new_place):
        o_r, o_c = place
        n_r, n_c = new_place
#копия массива
        new_matrix = deepcopy(self.matrix)

        new_matrix[n_r-1][n_c-1] = new_matrix[o_r-1][o_c-1]
        new_matrix[o_r-1][o_c-1] = self.matrix[n_r-1][n_c-1]

        return new_matrix

    def value_replaced(self, new_place):
        n_r, n_c = new_place
        return self.matrix[n_r-1][n_c-1]

    def distance(self, prev_place, new_place):
        x1, y1 = prev_place
        x2, y2 = new_place
        d = abs(x2 - x1) + abs(y2 - y1)

        return d
#Элементы не на своих местах 
    def count_dissimilar(self, matrix):
        dissmilar = 0
        count = 1
        for row in matrix:
            for col in row:
                if col and col is not count:
                    dissmilar += 1
#переход на сл.эл
                count += 1

        return dissmilar
#Генерация матрицы 4*4 
    def generate_matrix(self, initial_state):
        return [initial_state[i:i+4] for i in range(0, len(initial_state), 4)]

    def generate_numbers(self, grid_size):
        return [i for i in range(1, grid_size)] + [None]
#проверка на решение
#ИНВАРИАНТ
#чётность суммы двух следующих чисел, A и B. 
#Число A — это число пар плиток, в которых плитка с большим номером идёт перед плиткой с меньшим номером 
#Число B — это номер строки, в которой находится пустое поле.
    def is_solvable(self):
        self.place = self.get_block_place(block=None, matrix=self.matrix)
        result = self.sum_less_i() + self.b(block=None, matrix=self.matrix)

        return (result % 2 == 1)
# возвращает координаты пустоты
    def get_block_place(self, block, matrix): 
        for row in range(0, 4):
            for col in range(0, 4):
                if (matrix[row][col] is block):
                    return (row+1, col+1)
#Поиск значения А
    def sum_less_i(self):
        sumission = 0
        for num in range(0, 16):
#если пустота,то добавл все последующие элементы 
            if self.initial_state[num] is None:
                sumission += (16 - (num + 1))
                continue
            
            for current in range(num + 1, 16):
                if (self.initial_state[current] is None):
                    continue

                if (self.initial_state[num] > self.initial_state[current]):
                    sumission += 1

        return sumission
#Число В
    def b(self, block, matrix): 
        for row in range(0, 4):
            for col in range(0, 4):
                if (matrix[row][col] is block):
                    return (row+1)


In [2]:
#вводимые данные 
initial_state = [5, 1, 7, 3, 9, 2, 11, 4, 13, 6, 15, 8, None, 10, 14, 12]
#initial_state = [5, 8, 7, 11, 1, 6, 12, 2, 9, None, 13, 10, 14, 3, 4, 15]


#Задаем массив классу 
game = PuzzleGame(initial_state)

#вызов функц.генерация матрицы 4*4
game.matrix = game.generate_matrix(initial_state)
print(game)

#если пройдена проверка на решаемость 
if (game.is_solvable()):

    print("Решение найдено!")
#кол-во эл. не на местах
    dissimilars = game.count_dissimilar(game.matrix)
#кординаты пустоты 
    place = game.get_block_place(block=None, matrix=game.matrix)
    prev_move = None
    level = 0

#пока не все эл. на местах 
    while(dissimilars > 0):
        level += 1
#возможные перемещения
        moves = game.check_moves(prev_move)
        print("Перестановка номер {}: {}".format(level, moves))
        
        dissimilar = {}
#перемещ.по порядку 
        for move in moves:
#если не выходим за матрицу
            if (game.move_allowed(place, game.MOVE[move])):
#Новая координата пустоты
                new = game.new_place(place, game.MOVE[move])
#копируем матрицу
                new_matrix = game.apply_move(place, new)
#считаем не на своих местах
                dissimilars = game.count_dissimilar(new_matrix)
                dissimilar[move] = dissimilars

        print("Варианты передвижений : {}".format(dissimilar))
#выбираем с мин значением 
        dissimilars = min(dissimilar.values())
#список передвижений
        dissimilar_moves = list(dissimilar.keys())
#по разрешенному списку
        for move in dissimilar_moves:
#совпало с минимальным 
#то выбираем направление 
            if dissimilar[move] == dissimilars:
                prev_move = move

        

        print("Выбранное передвижение : {}".format(prev_move))
#Новая координата пустоты
        new_place = game.new_place(place, game.MOVE[prev_move])
#копируем матрицу
        game.matrix = game.apply_move(place, new_place)
        place = new_place
        print(game)
        

    print("Общее кол-во ходов = {}".format(level - 1))

else:
    print("Решение не найдено!")

[5, 1, 7, 3]
[9, 2, 11, 4]
[13, 6, 15, 8]
[None, 10, 14, 12]

Решение найдено!
Перестановка номер 1: ('UP', 'DOWN', 'LEFT', 'RIGHT')
Варианты передвижений : {'UP': 14, 'RIGHT': 15}
Выбранное передвижение : UP
[5, 1, 7, 3]
[9, 2, 11, 4]
[None, 6, 15, 8]
[13, 10, 14, 12]

Перестановка номер 2: ('UP', 'LEFT', 'RIGHT')
Варианты передвижений : {'UP': 13, 'RIGHT': 14}
Выбранное передвижение : UP
[5, 1, 7, 3]
[None, 2, 11, 4]
[9, 6, 15, 8]
[13, 10, 14, 12]

Перестановка номер 3: ('UP', 'LEFT', 'RIGHT')
Варианты передвижений : {'UP': 12, 'RIGHT': 13}
Выбранное передвижение : UP
[None, 1, 7, 3]
[5, 2, 11, 4]
[9, 6, 15, 8]
[13, 10, 14, 12]

Перестановка номер 4: ('UP', 'LEFT', 'RIGHT')
Варианты передвижений : {'RIGHT': 11}
Выбранное передвижение : RIGHT
[1, None, 7, 3]
[5, 2, 11, 4]
[9, 6, 15, 8]
[13, 10, 14, 12]

Перестановка номер 5: ('UP', 'DOWN', 'RIGHT')
Варианты передвижений : {'DOWN': 10, 'RIGHT': 11}
Выбранное передвижение : DOWN
[1, 2, 7, 3]
[5, None, 11, 4]
[9, 6, 15, 8]
[13, 10, 14, 1